In this notebook, we run an optimization making use of all of the functionality of horsetail matching. 

In [1]:
import numpy as np
import scipy.optimize as scopt

from horsetailmatching import HorsetailMatching, UniformParameter, GaussianParameter, IntervalParameter, UncertainParameter
from horsetailmatching.demoproblems import TP2
from horsetailmatching.surrogates import PolySurrogate

def fQOI(x, u):
    factor = np.sqrt(u[0]**2 + u[1]**2)
    q = 1 + (1 + factor*(x[0]**2 + x[1]*x[0] + x[1]**2))
    grad = [factor*(2*x[0] + x[1]), factor*(x[0] + 2*x[1])]
    return q, grad

def myPDF(q):
    if q > 1 or q < -1:
        return 0
    else:
        return 0.5

u_1 = UncertainParameter(myPDF, lower_bound=-1, upper_bound=1)
u_2 = GaussianParameter(mean=0, standard_deviation=1)
u_3 = IntervalParameter(lower_bound=-1, upper_bound=1)
             
def ftarget_u(h):
    return 0 - h**5

def ftarget_l(h):
    return 1 - h**5
             
qPolyChaos = PolySurrogate(dimensions=3, order=3, poly_type=['legendre', 'hermite', 'legendre'])
gradPolyChaos = [PolySurrogate(dimensions=3, order=3, poly_type=['legendre', 'hermite', 'legendre']),
                 PolySurrogate(dimensions=3, order=3, poly_type=['legendre', 'hermite', 'legendre'])]
u_quad_points = qPolyChaos.getQuadraturePoints()
             
def mySurrogateWithGrad(u_quad, q_quad, grad_quad):
    qPolyChaos.train(q_quad)
    for i, gPC in enumerate(gradPolyChaos):
        gPC.train(grad_quad[:, i])
    def qmodel(u):
        return qPolyChaos.predict(u)
    def gradmodel(u):
        return [gPC.predict(u) for gPC in gradPolyChaos]
    return qmodel, gradmodel

theHM = HorsetailMatching(fQOI, [u_1, u_2], jac=True, method='kernel', ftarget=(ftarget_u, ftarget_l),
                          samples_prob=1000, samples_int=50, integration_points=np.linspace(0, 50, 500),
                          surrogate=mySurrogateWithGrad, surrogate_points=u_quad_points, surrogate_jac=True,
                          verbose=True)

solution = scopt.minimize(theHM.evalMetric, x0=[3, 2], jac=True, method='BFGS')

----------
At design: [3 2]
Evaluating surrogate
Getting uncertain parameter samples
Evaluating quantity of interest at samples
Evaluating metric
Metric: 30.4549780761
Gradient: [11.730053580475126, 10.263796882915738]
----------
At design: [3 2]
Evaluating surrogate
Re-using stored samples
Evaluating quantity of interest at samples
Evaluating metric
Metric: 30.4549780761
Gradient: [11.730053580475126, 10.263796882915738]
----------
At design: [-8.73005358 -8.26379688]
Evaluating surrogate
Re-using stored samples
Evaluating quantity of interest at samples
Evaluating metric
Metric: 69.936329618
Gradient: [0.0, 0.0]
----------
At design: [ 0.0491101  -0.58202866]
Evaluating surrogate
Re-using stored samples
Evaluating quantity of interest at samples
Evaluating metric
Metric: 3.00815734701
Gradient: [-0.70646362908487859, -1.6280609448506482]
----------
At design: [-0.15231026  0.15741545]
Evaluating surrogate
Re-using stored samples
Evaluating quantity of interest at samples
Evaluating m

For other tutorials, see: http://www-edc.eng.cam.ac.uk/aerotools/horsetailmatching/